In [1]:
import dicom
import numpy as np
import os
from PIL import Image
import shutil
import scipy.misc
import random
from matplotlib import pyplot as plt
import SimpleITK as sitk
import json
from torchvision import transforms as T
from torchvision.transforms import functional as F
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import pandas as pd
import glob
import csv
def rm_mkdir(dir_path):
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print('Remove path - %s'%dir_path)
    os.makedirs(dir_path)
    print('Create path - %s'%dir_path)
def convertImage(raw):
    img = np.array(raw)
    img[img>255] = 255
    mean = np.mean(img)
    min_img = np.min(img)
    if min_img<-1500:
        min_img = min_img*(1.0/2.0)
    #print(mean, min_img)
    
    img = img - mean
    img = img*1.0 + mean*(mean/min_img)*1.1#修对比度和亮度
    img = img/np.max(img)
    #img[img>255] = 255
    img[img<0] = 0
    return img

/work/dependence/anaconda3/envs/py36/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
lis = glob.glob(os.path.join('./ISIC/dataset/cancer/B题-全部数据/数据集1/1001/arterial phase','*.dcm'))

In [ ]:
image = sitk.ReadImage('./ISIC/dataset/cancer/B题-全部数据/数据集1/1060/venous phase/20011.dcm')
image_array = sitk.GetArrayFromImage(image)
mask_image = Image.open('./ISIC/dataset/cancer/B题-全部数据/数据集1/1060/venous phase/20011_mask.png')
mask_image = np.array(mask_image)

print(np.max(mask_image))
plt.figure(figsize=(10,10))
img_t = 0.3*(mask_image/255.)*convertImage(image_array)[0,:,:]+convertImage(image_array)[0,:,:]
tmp_zeros = np.zeros(shape=(512,512,3))
tmp_zeros[:,:,0] = 0.3*(mask_image/255.)*convertImage(image_array)[0,:,:] + convertImage(image_array)[0,:,:]
tmp_zeros[:,:,1] = 0.2*(mask_image/255.)*convertImage(image_array)[0,:,:] + convertImage(image_array)[0,:,:]
tmp_zeros[:,:,2] = 0.05*(mask_image/255.)*convertImage(image_array)[0,:,:] + convertImage(image_array)[0,:,:]
plt.imshow(tmp_zeros)

In [ ]:
plt.imshow(convertImage(image_array)[0,:,:],'gray')

In [ ]:
tmp_img = Image.open('./ISIC/dataset/cancer/B题-全部数据/数据集1/1001/venous phase/20011_mask.png')
Loc = (100, 170, 420, 490)
tmp_img

In [ ]:
plt.imshow(image_array[0,170:490,100:420], 'gray')

negative : positive = 64 : 45

In [ ]:
a = np.array(df.loc[df.ID==1108, 'label'])[0]
if a=='-':
    tmp_label = 'negative'
else:
    tmp_label = 'positive'
print(tmp_label)

In [4]:
Paths = './ISIC/dataset/cancer/B题-全部数据/数据集1/'
frames_save_path = './ISIC/five-video-classification-methods-master/data/cancer/'

list_Paths = os.listdir(Paths)
random.shuffle(list_Paths)

df = pd.read_csv('./ISIC/dataset/cancer/B题-全部数据/label2.csv', encoding = "ISO-8859-1")
df = pd.DataFrame(df)
def convertImage(raw):
    img = np.array(raw)
    img[img>255] = 255
    mean = np.mean(img)
    min_img = np.min(img)
    if min_img<-1500:
        min_img = min_img*(1.0/2.0)
    #print(mean, min_img)
    
    img = img - mean
    img = img*1.0 + mean*(mean/min_img)*1.1#修对比度和亮度
    img = img/np.max(img)
    #img[img>255] = 255
    img[img<0] = 0
    return img

In [5]:
def data_augmentation(raw_path):
    image = Image.open(raw_path)
    
    aspect_ratio = image.size[1]/image.size[0]
    hw = (image.size[0],image.size[1])
    Transform = []

    ResizeRange = random.randint(300,320)
    Transform.append(T.Resize((int(ResizeRange*aspect_ratio),ResizeRange)))
    p_transform = random.random()

    if p_transform <= 0.6:
        
        RotationRange = random.randint(-15,15)
        Transform.append(T.RandomRotation((RotationRange,RotationRange)))
        CropRange = random.randint(250,270)
        Transform.append(T.CenterCrop((int(CropRange*aspect_ratio),CropRange)))
        Transform = T.Compose(Transform)

        image = Transform(image)

        ShiftRange_left = random.randint(0,20)
        ShiftRange_upper = random.randint(0,20)
        ShiftRange_right = image.size[0] - random.randint(0,20)
        ShiftRange_lower = image.size[1] - random.randint(0,20)
        image = image.crop(box=(ShiftRange_left,ShiftRange_upper,ShiftRange_right,ShiftRange_lower))

        if random.random() < 0.5:
            image = F.hflip(image)

        Transform =[]


    Transform.append(T.Resize((hw[1], hw[0])))
    Transform = T.Compose(Transform)

    image = Transform(image)

    return image, hw, p_transform

In [20]:
trainframes_save_path = './ISIC/five-video-classification-methods-master/data/cancer/train/'
testframes_save_path = './ISIC/five-video-classification-methods-master/data/cancer/test/'
#rm_mkdir(frames_save_path)
rm_mkdir(trainframes_save_path+'/negative/')
rm_mkdir(trainframes_save_path+'/positive/')
rm_mkdir(testframes_save_path+'/negative/')
rm_mkdir(testframes_save_path+'/positive/')
Loc = (170, 490, 100, 420)
def convert_frames(root_paths, list_root_paths, record, frame_train, frame_test, loc, ratio = 0.7):
    count = 0
    data_file = []
    tictoc =0 
    for i in list_root_paths:
        tmp_label = np.array(record.loc[record.ID==int(i), 'label'])[0]
        if tmp_label=='-':
            tmp_label = 'negative'
        else:
            tmp_label = 'positive'
        list_period_paths = os.listdir(root_paths+i+'/')
        for j in list_period_paths:
            count+=1
            countmask = 0
            list_patient_paths = os.listdir(root_paths+i+'/'+j+'/')
            dcm_filenames = glob.glob(os.path.join(root_paths+i+'/'+j+'/', '*.dcm'))
            dcm_filenames = sorted(dcm_filenames)
            mask_filenames = glob.glob(os.path.join(root_paths+i+'/'+j+'/', '*.png'))
            mask_filenames = sorted(mask_filenames)
            #for m, pngname in dcm_filenames, mask_filenames:
            for k in range(len(dcm_filenames)):
                m = dcm_filenames[k]
                pngname = mask_filenames[k]
                mask_img = Image.open(pngname)
                mask_img = np.array(mask_img)
                if k+1<len(dcm_filenames):
                    
                    pngname_next = mask_filenames[k+1]
                    mask_img_next = Image.open(pngname_next)
                    mask_img_next = np.array(mask_img_next)
                else:
                    #pngname_next = mask_filenames[k]
                    #mask_img_next = Image.open(pngname)
                    mask_img_next = mask_img
                if (np.max(mask_img)>0) or (np.max(mask_img_next)>0):
                    countmask+=1
                    tmp_zeros = np.zeros(shape=(512,512,3))
                    #mask_image = np.array(mask_image)
                    image = sitk.ReadImage(m)
                    image_array = sitk.GetArrayFromImage(image)
                    dcm_image = image_array[0,:,:]
                    convertImg = convertImage(dcm_image)
                    
                    
                    tmp_zeros[:,:,0] = 0.3*(mask_img/255.)*convertImg[:,:] + convertImg[:,:]
                    tmp_zeros[:,:,1] = 0.2*(mask_img/255.)*convertImg[:,:] + convertImg[:,:]
                    tmp_zeros[:,:,2] = 0.05*(mask_img/255.)*convertImg[:,:] + convertImg[:,:]
                    
                    imgRevert = tmp_zeros[loc[0]:loc[1],loc[2]:loc[3],:]
        
                    #imgRevert = np.repeat(np.expand_dims(convertImg[loc[0]:loc[1],loc[2]:loc[3]],
                    #                                     axis=2), 3, axis=2)
                    #imgRevert
                    m_split = m.split('/')[-1].split('.')[0]

                    if count<175:
                        scipy.misc.imsave(frame_train+tmp_label+'/'+i+'_'+j[0]+'_au0-'+m_split+'.jpg', imgRevert)
                        au1, _, _ = data_augmentation(frame_train+tmp_label+'/'+i+'_'+j[0]+'_au0-'+m_split+'.jpg')
                        #au1 = au1.crop(box=(loc[1], loc[0], loc[3], loc[2]))
                        au1.save(frame_train+tmp_label+'/'+i+'_'+j[0]+'_au1-'+m_split+'.jpg')
                        
                    else:
                        scipy.misc.imsave(frame_test+tmp_label+'/'+i+'_'+j[0]+'_au3-'+m_split+'.jpg', imgRevert)
                        au2, _, _ = data_augmentation(frame_test+tmp_label+'/'+i+'_'+j[0]+'_au3-'+m_split+'.jpg')
                        #au2 = au2.crop(box=(loc[1], loc[0], loc[3], loc[2]))
                        au2.save(frame_train+tmp_label+'/'+i+'_'+j[0]+'_au2-'+m_split+'.jpg')
            if count<175:
                data_file.append(['train', tmp_label, i+'_'+j[0]+'_au0', countmask])
                data_file.append(['train', tmp_label, i+'_'+j[0]+'_au1', countmask])
            else:
                data_file.append(['test', tmp_label, i+'_'+j[0]+'_au3', countmask])
                data_file.append(['train', tmp_label, i+'_'+j[0]+'_au2', countmask])
           
        print(tmp_label, i, m_split, ' handling process is {}/{}'.format(tictoc, len(list_root_paths)))
        tictoc+=1 
    
    with open(frames_save_path+'data_file.csv', 'w') as fout:
        writer = csv.writer(fout)
        writer.writerows(data_file)
    return data_file

Remove path - ./ISIC/five-video-classification-methods-master/data/cancer/train//negative/
Create path - ./ISIC/five-video-classification-methods-master/data/cancer/train//negative/
Remove path - ./ISIC/five-video-classification-methods-master/data/cancer/train//positive/
Create path - ./ISIC/five-video-classification-methods-master/data/cancer/train//positive/
Remove path - ./ISIC/five-video-classification-methods-master/data/cancer/test//negative/
Create path - ./ISIC/five-video-classification-methods-master/data/cancer/test//negative/
Remove path - ./ISIC/five-video-classification-methods-master/data/cancer/test//positive/
Create path - ./ISIC/five-video-classification-methods-master/data/cancer/test//positive/


In [21]:
DATA_FILE = convert_frames(Paths, list_Paths, df, trainframes_save_path, testframes_save_path, Loc)

/work/dependence/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:65: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


positive 1052 20025  handling process is 0/107
positive 1069 20014  handling process is 1/107
negative 1034 20017  handling process is 2/107
negative 1036 20016  handling process is 3/107
positive 1038 20017  handling process is 4/107
negative 1053 20020  handling process is 5/107
negative 1067 20033  handling process is 6/107
positive 1008 20017  handling process is 7/107
positive 1070 20015  handling process is 8/107
positive 1103 20016  handling process is 9/107
negative 1055 20023  handling process is 10/107
negative 1063 20039  handling process is 11/107
positive 1068 20018  handling process is 12/107
negative 1010 20016  handling process is 13/107
positive 1072 20017  handling process is 14/107
negative 1058 20011  handling process is 15/107
positive 1071 20017  handling process is 16/107
negative 1011 20017  handling process is 17/107
negative 1056 20021  handling process is 18/107
negative 1001 20016  handling process is 19/107
negative 1066 20015  handling process is 20/107
ne

/work/dependence/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:71: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


negative 1107 20018  handling process is 87/107
negative 1099 20014  handling process is 88/107
negative 1090 20015  handling process is 89/107
negative 1026 20019  handling process is 90/107
positive 1043 20015  handling process is 91/107
negative 1012 20025  handling process is 92/107
negative 1028 20012  handling process is 93/107
negative 1032 20022  handling process is 94/107
positive 1061 20024  handling process is 95/107
positive 1095 20010  handling process is 96/107
positive 1105 20015  handling process is 97/107
positive 1004 20025  handling process is 98/107
positive 1019 20017  handling process is 99/107
positive 1060 20017  handling process is 100/107
negative 1082 20015  handling process is 101/107
negative 1039 20023  handling process is 102/107
negative 1087 20022  handling process is 103/107
positive 1037 20021  handling process is 104/107
positive 1035 20021  handling process is 105/107
negative 1094 20016  handling process is 106/107


In [22]:
rm_mkdir('./ISIC/five-video-classification-methods-master/data/train/')
rm_mkdir('./ISIC/five-video-classification-methods-master/data/test/')
! cp -r ./ISIC/five-video-classification-methods-master/data/cancer/train ./ISIC/five-video-classification-methods-master/data/

! cp -r ./ISIC/five-video-classification-methods-master/data/cancer/test ./ISIC/five-video-classification-methods-master/data/
! cp -r ./ISIC/five-video-classification-methods-master/data/cancer/data_file.csv ./ISIC/five-video-classification-methods-master/data/

Remove path - ./ISIC/five-video-classification-methods-master/data/train/
Create path - ./ISIC/five-video-classification-methods-master/data/train/
Remove path - ./ISIC/five-video-classification-methods-master/data/test/
Create path - ./ISIC/five-video-classification-methods-master/data/test/
